In [1]:
from cache import cache
from collections import defaultdict
from decimal import Decimal
import httpx
import json
import pandas as pd
from pprint import pprint
import sys
from tabulate import tabulate
import time
from web3 import Web3

from IPython.display import display, HTML

In [2]:
# Gas Price - we could use web3 but this is fine
GAS_URL = 'https://api.debank.com/chain/gas_price_dict_v2?chain=eth'
c = cache.get(GAS_URL, refresh=True)
gas = json.loads(c['value'])
gas_price = gas['data']['normal']['price']/10**18

# ETH price
ETH_PRICE_URL = 'https://api.coingecko.com/api/v3/simple/price?vs_currencies=usd&ids=ethereum'
c = cache.get(ETH_PRICE_URL, refresh=True)
eth = json.loads(c['value'])
price_eth = eth['ethereum']['usd']

print(f'ETH Price: ${price_eth:,.2f}')
print(f'Gwei: {gas_price*10**9:,.1f}')

ETH Price: $2,748.20
Gwei: 46.0


In [3]:
def print_cost(title, data):
    # Calculate gas cost
    total_eth = 0.0
    total_usd = 0.0
    for d in data:
        cost_eth = d['gas'] * gas_price
        total_eth += cost_eth
        cost_usd = cost_eth * price_eth
        total_usd += cost_usd
        d['cost_eth'] = cost_eth
        d['cost_usd'] = cost_usd

    # Add Total
    data.append({
        'description': 'TOTAL',
        'method': None,
        'gas': None,
        'contract': None,
        'cost_eth': total_eth,
        'cost_usd': total_usd,
    })
    
    # Contracts
    contracts = {
        '0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0': 'FXS',
        '0xc8418af6358ffdda74e09ca9cc3fe03ca6adc5b0': 'veFXS',
        '0xc6764e58b36e26b08Fd1d2AeD4538c02171fA872': 'veFXSYieldDistributorV4',
        '0x3669C421b77340B2979d1A00a792CC2ee0FcE737': 'FraxGaugeControllerV2',
        '0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0': 'FraxFarm_UniV3_veFXS_FRAX_USDC',
    }
    
    for d in data:
        if d['contract'] in contracts:
            address = d['contract']
            name = contracts[address] 
            d['contract'] = f'<a href="https://etherscan.io/address/{address}">{name}</a>'
    
    # Print a nice table
    display(HTML(f'<b>{title}</b>')) 
    colalign = ('left', 'left', 'left', 'right', 'right', 'right')
    floatfmt = (None, None, None, None, ".5f", ".2f")
    display(tabulate(data, headers='keys', tablefmt='unsafehtml', colalign=colalign, floatfmt=floatfmt))

In [4]:
# veFXS Staking 
title = 'Staking veFXS (First Time)'
data = [
    {
        'description': 'Approve FXS', 
        'contract': '0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0',
        'method': 'approve', 
        'gas': 46677
    },
    {
        'description': 'Lock Tokens', 
        'contract': '0xc8418af6358ffdda74e09ca9cc3fe03ca6adc5b0',
        'method': 'create_lock', 
        'gas': 442770
    },
    {
        'description': 'Checkpoint', 
        'contract': '0xc6764e58b36e26b08Fd1d2AeD4538c02171fA872',
        'method': 'checkpoint', 
        'gas': 200000
    },
]
print_cost(title, data)

description,contract,method,gas,cost_eth,cost_usd
Approve FXS,FXS,approve,46677,0.00215,5.90
Lock Tokens,veFXS,create_lock,442770,0.02037,55.97
Checkpoint,veFXSYieldDistributorV4,checkpoint,200000,0.00920,25.28
TOTAL,,,,0.03171,87.16


In [5]:
# Claiming veFXS Rewards 
title = 'Claiming veFXS Rewards'
data = [
    {
        'description': 'Claim Rewards', 
        'contract': '0xc6764e58b36e26b08Fd1d2AeD4538c02171fA872',
        'method': 'getYield', 
        'gas': 230000
    },
]
print_cost(title, data)

description,contract,method,gas,cost_eth,cost_usd
Claim Rewards,veFXSYieldDistributorV4,getYield,230000,0.01058,29.08
TOTAL,,,,0.01058,29.08


In [6]:
# Voting for a Gauge
title = 'Voting for a Gauge (single vote; 2 calls to re-assign)'
data = [
    {
        'description': 'Gauge Voting', 
        'contract': '0x3669C421b77340B2979d1A00a792CC2ee0FcE737',
        'method': 'vote_for_gauge_weights', 
        'gas': 250000
    },
]
print_cost(title, data)

description,contract,method,gas,cost_eth,cost_usd
Gauge Voting,FraxGaugeControllerV2,vote_for_gauge_weights,250000,0.01150,31.60
TOTAL,,,,0.01150,31.60


In [7]:
# Claiming from Gauge Farm
title = 'Claiming from Gauge Farm (UniV3 FRAX/USDC; single lock)'
data = [
    {
        'description': 'Claim Rewards', 
        'contract': '0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0',
        'method': 'vote_for_gauge_weights', 
        'gas': 400000
    },
]
print_cost(title, data)

description,contract,method,gas,cost_eth,cost_usd
Claim Rewards,FraxFarm_UniV3_veFXS_FRAX_USDC,vote_for_gauge_weights,400000,0.01840,50.57
TOTAL,,,,0.01840,50.57


# Governance
Snapshot voting is gasless (FREE!)

Vote here if you have veFXS: https://snapshot.org/#/frax.eth/

Forums are also free:
https://gov.frax.finance/